## Introducción e hipótesis

El objetivo es predecir si un cliente se dará de baja o no de la plataforma.

In [ ]:
#Las librerías utilizadas en este documento son:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn import metrics
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
sns.set()

## Carga de datos

In [ ]:
data  = pd.read_csv('./data/Datos ML 2021 Q2.csv', sep=';')
print("El dataset tiene {} filas y {} columnas".format(data.shape[0], data.shape[1]))
data.head()

## Descripción del dataset

#### El dataset tiene las siguientes columnas:

- CustomerID: ID del cliente
- Churn: Columna que indica si el cliente dejó de usar la plataforma o no. 1 es que se da de baja.
- CustomerTenure: Es el tiempo transcurrido desde el inicio de la relación con el cliente (en meses)
- MainDeviceLogin: Dispositivo principal que utiliza el cliente para acceder a la plataforma
- CityTier: Indicador del nivel de desarrollo de la ciudad donde vive el cliente
- WarehouseToHome: Distancia desde el centro de distribución a la vivienda del cliente (en km)
- MainPaymentMode: Método de pago más utilizado por el cliente
- Gender: Género del cliente
- HourSpendOnApp: Número de horas que el cliente ha pasado en la plataforma
- DeviceRegistered: Número de dispositivos en los que el cliente ha accedido a la plataforma
- PrefCategory: Categoría más común de las compras del cliente en el último mes
- SatisfactionScore: Nivel de satisfacción del cliente con el servicio
- MaritalStatus: Estado civil del cliente
- NumberOfAddress: Número de direcciones diferentes registradas por el cliente
- Complain: Si ha realizado reclamos
- OrderAmountHikeFromlastYear: Incremento porcentual en la cantidad de compras con respecto al año anterior
- CouponUsed: Número de cupones usados en el último mes
- OrderCount: Número de compras realizadas en el último mes
- DaySinceLastOrder: Cantidad de días desde la última compra
- CashbackAmount: Promedio de reembolsos pedidos en el último mes

## Exploración de datos

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isna().sum()

In [ ]:
data['Churn'].value_counts()

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(data.corr(), annot=True, vmin=-1, cmap='Blues')

In [ ]:
#sns.pairplot(data)

Armar histogramas de frecuencia del churn vs cada variable

In [ ]:
data_without_target = data.drop(['Churn'], axis=1)

In [ ]:
fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(32,32))
fig.suptitle('Histogramas normalizados')
for c, ax in zip(data.columns, axes.flatten()):
    sns.histplot(data = data.loc[data['Churn']==0, c].dropna(), stat = 'density', ax = ax, kde = False )
    sns.histplot(data = data.loc[data['Churn']==1, c].dropna(), stat = 'density', kde=False, ax=ax, color = 'orange')
    ax.legend(['Churn = 0', 'Churn = 1'])

## Limpieza de datos

In [ ]:
data_clean = data.dropna()
data_clean.shape

## Preparación de datos

### Variables categóricas (dummies y ordinales)

### Variables numéricas (normalización?)

## Separación de datos

## En cada modelo ver los distintos indicadores y decidir cuál es el más apropiado para el dataset

## KNN

## Regresión logística

## Naive Bayes

## Conclusiones